This notebook is intended to support supervision of the training procuedure of regression models (previously performed by means of scripts blabkbox_model_training.py, ES_model_training.py and semiPar_model_training.py) by providing visual elaboration of the performance metrics, predictions and residual errors of such models. Besides, the internal architecture of models is visualized. 

This notebook provides quick insights into the training procedure, to make ensure everything is going smoothly, while a more in depth analysis of regression models predictions and performance, and a comparative of the different approaches, is available in notebook models_comparison.ipynb

In [ ]:
from termcolor import colored
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools 
import seaborn as sns
import keras
import tensorflow as tf

from pathlib import Path
import sys
PROJECT_DIR =Path(os.path.abspath('')).parents[1]
sys.path.append(os.fspath(PROJECT_DIR))
#sys.path.insert(0,'../')  # add previous directory to path to load constants module

#from MLP_fx import create_model, 
from pipeline.regression_fx import realElbows
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score


from pipeline.definitions import *

import random

### Select graphic settings

In [ ]:
graph_setting="notebook" #or "article"

In [ ]:
if graph_setting=="article":
    
    #journal-quality parameter settings
    resolution_factor=2
    desired_font=10

elif graph_setting=="notebook":
    resolution_factor=1
    desired_font=12
    
#conversion factors
cm_to_inch=0.393701
classic_proportion=6.4/4.8
golden_rate=1.618

#Elsevier column width is 8.4 cm, double-column width is 17.7 cm (in inches: 3.31 and 6.97)
small_figsize=(resolution_factor*3.31, resolution_factor*3.31/classic_proportion)
big_figsize=(resolution_factor*6.97, resolution_factor*6.97/classic_proportion)

#changings regarding fonttypex
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = "Arial"

font_size=resolution_factor*desired_font


#define path for figures
figures_path=FIGURES
#check existance of figure path
if not os.path.exists(figures_path):
    print("The selected directory to store figures does not exist")

## Load data, models and scalers

In [ ]:
# Import a case study dataset using pandas
method_short="ts" #that is, time serie, or ew, that is elementWise
dataset = pd.read_csv(os.path.join(DATA_NORMALIZED, 'norm_'+method_short+'_buildings_dataset.csv'))
print("Shape of dataset: "+str(dataset.shape))
#load dataset description
data_info=pd.read_csv(DATASETS+'/buildings_data_description.csv')

In [ ]:
IDs=dataset.ID.unique()
models_name_list=["Parametric", "Semi-Parametric", "Non-Parametric"]
approaches=["uniES", "semiPar",  "blackbox"]
models_list=['P_phys', 'P_semiPar', 'P_blackbox']
##load scalers
file_name="scalers_"+method_short
with open(DATA+"/"+file_name+".pkl", 'rb') as f:
    scalers_dict = pickle.load(f)
#create a models dictionary (keys: approach, casestudy, model_n)
models={}
#create a dictionary for the predictions (keys: approach, casestudy, model_n)
predictions={}
NN_predictions={}
#create dataframes to store performance metrics from the models
df_r2=pd.DataFrame(index=pd.MultiIndex.from_product([models_name_list, ['Train', 'Test']]), columns=pd.MultiIndex.from_product([IDs.tolist(), [ "model_"+str(x) for x in range(10) ]]))
df_mape=df_r2.copy()

In [ ]:
#prepare input matrices for the different approaches
ind_var=["T_a", "RH", "Wv", "atmP", "G", "s_Wa", "c_Wa", "s_H", "c_H", "s_D", "c_D", "dayType"]
n_inputs=len(ind_var)
modeled_ind_var="T_a"
dep_var="P"

### Load Models

In [ ]:
for approach, models_path in zip(approaches, [PAR_MODELS, SEMIPAR_MODELS, BLACKBOX_MODELS]):
    approach_models=[f for f in listdir(models_path) if isfile(join(models_path, f))]
    for caseStudy in IDs:
        caseStudy_models_list=[f for f in approach_models if caseStudy in f]
        for model, model_n in zip(caseStudy_models_list, [ "model_"+str(x) for x in range(len(caseStudy_models_list)) ]):
            if (approach=="semiPar") | (approach=="uniES"):
                models[approach, caseStudy, model_n]=tf.keras.models.load_model(models_path+'/'+model, custom_objects={'realElbows':realElbows}, compile=False)
            else:
                models[approach, caseStudy, model_n]=tf.keras.models.load_model(models_path+'/'+model)

### Check the architeture of the semiPar model

In [ ]:
caseStudy='building_1001'
keras.utils.plot_model(models["semiPar", caseStudy, model_n], show_shapes=True)

### Check number of models

In [ ]:
for approach, approach_name, n_required in zip(approaches, models_name_list, [len(IDs), len(IDs), len(IDs)]):
    approach_models_names=[model for model in models if approach in model]
    n_models=len(approach_models_names)
    if n_models==n_required:
        print(colored("SUCCESSFUL MODEL LOADING: "+str(n_required)+" "+approach_name+" models have been loaded", "green"))
    elif n_models<n_required:
        print(colored("ERROR - MODELS MISSING: "+str(n_models)+" "+approach_name+" models have been loaded, but "+str(n_required)+" models were expected", "red"))
    else:
        print(colored("ERROR - TOO MANY MODELS: "+str(n_models)+" "+approach_name+" models have been loaded, but "+str(n_required)+" models were expected", "red"))

## Models performance assessment

#### Produce and store predictions

In [ ]:
for approach, approach_name in zip(approaches, models_name_list):
    #approach_models=[f for f in allmodels if approach in f]
    for caseStudy in IDs:
        #caseStudy_models_list=[f for f in approach_models if caseStudy in f]
        
        #define modeled input
        df=dataset.loc[dataset.ID==caseStudy]
        x=pd.DataFrame(df[ind_var])
        x_modeled=pd.DataFrame(df[modeled_ind_var])
        y=pd.DataFrame(df[dep_var])
        
        #split test and train 
        if method_short== "ts":
            x_train, x_test, y_train,y_test= train_test_split(x, y, test_size=0.33, shuffle=False)
        elif method_short== "ew":
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

        x_modeled_train=pd.DataFrame(x_train[modeled_ind_var])
        x_modeled_test=pd.DataFrame(x_test[modeled_ind_var])

        #check number of models to iterate for
        approach_models=[model for model in models if approach in model]
        caseStudy_models_list=[f for f in approach_models if caseStudy in f]
        n_models=len(caseStudy_models_list)
        
        for n, model_n in zip(range(n_models), [ "model_"+str(x) for x in range(n_models) ]):
            model=models[approach, caseStudy, model_n]
            if approach=="uniES":
                y_NN_pred=model.predict(x_modeled, verbose=0)

            elif approach=="semiPar":
                y_NN_pred=model.predict((x_modeled, x), verbose=0)

            elif approach=="blackbox":
                y_NN_pred=model.predict(x, verbose=0)

            #split test and train 
            if method_short== "ts":
                 _,_, y_NN_pred_train, y_NN_pred_test= train_test_split(x, y_NN_pred, test_size=0.33, shuffle=False)
            elif method_short== "ew":
                 _,_, y_NN_pred_train, y_NN_pred_test = train_test_split(x, y_NN_pred, test_size=0.33, random_state=42)
        
            # rescale real outputs
            y_real=scalers_dict[caseStudy, "y"].inverse_transform(y)
            y_real_train=scalers_dict[caseStudy, "y"].inverse_transform(y_train)
            y_real_test=scalers_dict[caseStudy, "y"].inverse_transform(y_test)
            
            #predictions of the full set
            y_pred=scalers_dict[caseStudy, "y"].inverse_transform(y_NN_pred)
            predictions[approach, caseStudy, model_n]=y_pred

            #store non-rescaled predictions
            NN_predictions[approach, caseStudy, model_n, "Train"]=y_NN_pred_train
            NN_predictions[approach, caseStudy, model_n, "Test"]=y_NN_pred_test
            NN_predictions[approach, caseStudy, model_n, "Whole Set"]=y_NN_pred
            
            
            #train set predictions and scores
            y_pred_train=scalers_dict[caseStudy, "y"].inverse_transform(y_NN_pred_train)
            df_r2.loc[approach_name, 'Train'][caseStudy, model_n] =r2_score(y_real_train, y_pred_train)
            df_mape.loc[approach_name, 'Train'][caseStudy, model_n]=mean_absolute_percentage_error(y_real_train, y_pred_train)
            
            #test set predictions and scores
            y_pred_test=scalers_dict[caseStudy, "y"].inverse_transform(y_NN_pred_test)
            df_r2.loc[approach_name, 'Test'][caseStudy, model_n] =r2_score(y_real_test, y_pred_test)
            df_mape.loc[approach_name, 'Test'][caseStudy, model_n]=mean_absolute_percentage_error(y_real_test, y_pred_test)
            
    print(colored("Predictions from the "+approach_name+" models correctly retrieved", "green"))

#### Performance indexes overview

In [ ]:
#initialize graphical object
fig, (ax1, ax2) = plt.subplots(2, 3, figsize=big_figsize)
subplot_pos_list=[231, 232, 233, 234, 235, 236]

for case, subplot_pos in zip(list(itertools.product([df_r2, df_mape], models_name_list)), subplot_pos_list):
    df=case[0]
    df_test=pd.DataFrame(df.loc[(case[1], "Test"), :]).unstack(level=1).T.astype(float)
    #access to specific subplot
    ax=plt.subplot(subplot_pos)

    sns.boxplot(data=df_test)#, y='variable', x='value', hue='group', palette=colors)
    
    if subplot_pos in [231, 232, 233]:  
        ax.set_title(case[1], fontsize=font_size)
        ax.set_ylim([0, 1])
    else: 
        ax.set_ylim([0, 0.035])
    if subplot_pos==231:
        ax.set_ylabel("$R^2$", fontsize=font_size-2, rotation=0)
        #ax.yaxis.set_label_coords(-0.25, +0.5)
    elif subplot_pos==234:
        ax.set_ylabel("MAPE", fontsize=font_size-2, rotation=0)
        #ax.yaxis.set_label_coords(-0.25, +0.5)
    ax.set_xticklabels([])
    if subplot_pos in [232, 233, 235, 236]:
       ax.set_yticklabels([]) 
    ax.grid()
    ax.tick_params(labelsize=font_size)
plt.tight_layout()

### Best Model Predictions comparison

In [ ]:
n_plots=2
for  caseStudy in IDs[:n_plots]:
    #define modeled input
    df=dataset.loc[dataset.ID==caseStudy]
    y_real=df[dep_var]
    
    if method_short== "ts":
        _, _, y_real_train, y_real_test = train_test_split(x, y_real, test_size=0.33, shuffle=False)
    elif method_short== "ew":
        _,_ , y_real_train, y_real_test = train_test_split(x, y_real, test_size=0.33, random_state=42)
    
    #initialize a new graphical object
    fig, axs = plt.subplots(1, 3, figsize=(big_figsize[0], big_figsize[1]/2) )
    
    for approach, approach_name, ax in zip(approaches, models_name_list, axs):

        #check number of models to iterate for
        approach_models=[model for model in models if approach in model]
        caseStudy_models_list=[f for f in approach_models if caseStudy in f]
        n_models=len(caseStudy_models_list)
        
        n_models=1 #set n_models=1 if you want only the best SemiPar and NonPar model to be displayed, leave it as it was if you want to see all the models
        
        for n, model_n in zip(range(n_models), [ "model_"+str(x) for x in range(n_models) ]):
            y_pred_train=NN_predictions[approach, caseStudy, model_n, "Train"]
            y_pred_test=NN_predictions[approach, caseStudy, model_n, "Test"]
            #scatter predictions
            ax.scatter(y_real_train, y_pred_train, 1.5, alpha=0.2)
            ax.scatter(y_real_test, y_pred_test, 1.5, alpha=0.2)
            ax.plot([0, 1], [0, 1], color="black", linestyle='--')

            #graphical custom options
            ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
            ax.tick_params(labelsize=font_size-2)
            ticks=[0, 0.25, 0.5, 0.75, 1]
            ax.set_xticks(ticks)
            ax.set_yticks(ticks)

            if caseStudy!=IDs[n_plots-1]:
                ax.tick_params(axis='x',which='both',length=0.1,width=0.1,pad=1)
                #ax.set_xticklabels([])

            if caseStudy==IDs[0]:
                ax.set_title(approach_name, fontsize=font_size)

                
            #arrange image details (ticks, tickslabels and titles)
            if ax!=axs[0]:
                    ax.tick_params(axis='y',which='both',length=0.1,width=0.1,pad=1)
                    ax.set_yticklabels([])

            text_y=fig.supylabel(caseStudy+'\n\n'+'Predictions [-]', x=0.02, fontsize=font_size)     
            plt.tight_layout()
            
    if caseStudy==IDs[0]:
        axs[2].legend(["Train", "Test"], fontsize=font_size)
 
text_x=fig.supxlabel('True Values [-]', y=-0.02, fontsize=font_size)


### Check residuals

In [ ]:
n_plots=2

for  caseStudy in IDs[:n_plots]:
    #define modeled input
    df=dataset.loc[dataset.ID==caseStudy]
    y_real=df[dep_var].values
 
    #initialize a new graphical object
    fig, axs = plt.subplots(1, 3, figsize=(big_figsize[0], big_figsize[1]/2))
    font_size=12
    
    for approach, approach_name, ax in zip(approaches, models_name_list, axs):
        y_pred=NN_predictions[approach, caseStudy, "model_0", "Whole Set"].flatten()
        res = y_real-y_pred
        ax.hist(res, bins=30)

        ax.tick_params(labelsize=font_size-2)
        #xticks=[-0.5, -0.25 ,0, 0.25, 0.5]
        #yticks=[0, 500,  1000, 1500, 2000]
        #ax.set_xticks(xticks)
        #ax.set_yticks(yticks)

        #graphical custom options
        if caseStudy!=IDs[n_plots-1]:
            ax.tick_params(axis='x',which='both',length=0.1,width=0.1,pad=1)
            #ax.set_xticklabels([])

        if ax!=axs[0]:
                ax.tick_params(axis='y',which='both',length=0.1,width=0.1,pad=1)
                #ax.set_yticklabels([])
            
        if caseStudy==IDs[0]:
                ax.set_title(approach_name, fontsize=font_size)
            
        text_y=fig.supylabel(caseStudy+'\n\n'+'Number of occurences', x=-0.02, fontsize=font_size)   

    plt.tight_layout()
text_x=fig.supxlabel('Error (calculated from normalized values)', y=-0.05, fontsize=font_size)

### Analysis of correlation of residuals 

In [ ]:
#select one case study
caseStudy=random.choice(IDs)
caseStudy=IDs[0]
df=dataset.loc[dataset.ID==caseStudy]

#add to the matrix the output values (predicted power) and the residuals from the different models
df_extended=df.copy()
df_extended['P^_ES']=NN_predictions["uniES", caseStudy, "model_0", "Whole Set"]
df_extended['P^_semi']=NN_predictions["semiPar", caseStudy, "model_0", "Whole Set"]
df_extended['P^_bb']=NN_predictions["blackbox", caseStudy, "model_0", "Whole Set"]
df_extended['err_ES']=df_extended["P"]-df_extended["P^_ES"]
df_extended['err_semi']=df_extended["P"]-df_extended["P^_semi"]
df_extended['err_bb']=df_extended["P"]-df_extended["P^_bb"]

#reorder dataframe columns
new_order=['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb', 'T_a', 'RH', 'Wv', 'atmP', 'G', 's_Wa', 'c_Wa', 's_H', 'c_H',
       's_D', 'c_D', 'dayType', 'ID']
df_extended=df_extended[new_order]

#calculate correlation
df_corr=df_extended.drop("ID", axis=1).corr()

#select features to be displayed in the reduced correlation heat map
reduced_df_corr=df_corr.loc[['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb'], ['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb', 'T_a', 'RH', 'Wv', 'atmP', 'G', 's_Wa', 'c_Wa', 's_H', 'c_H',
       's_D', 'c_D', 'dayType']]

#provide one correlation heatmap with labels and colorbar
fig, axs=plt.subplots(1, figsize=small_figsize)
axs.tick_params(labelsize=font_size-2)
sns.heatmap(df_corr, vmin=0.0, vmax=1.0, ax=axs, square=True)
plt.title("Sample correlation Matrix", fontsize=font_size)

In [ ]:
#display all the subcases, without labels and colormpap
plots_per_row=2
fig, axs=plt.subplots(int(len(IDs)/plots_per_row), plots_per_row, figsize=(4*plots_per_row, 1.2*len(IDs)))
for caseStudy, ax, i in zip(IDs, axs.flatten(), range(len(IDs))):
    df=dataset.loc[dataset.ID==caseStudy]

    #add to the matrix the output values (predicted power) and the residuals from the different models
    df_extended=df.copy()
    df_extended['P^_ES']=NN_predictions["uniES", caseStudy, "model_0", "Whole Set"]
    df_extended['P^_semi']=NN_predictions["semiPar", caseStudy, "model_0", "Whole Set"]
    df_extended['P^_bb']=NN_predictions["blackbox", caseStudy, "model_0", "Whole Set"]
    df_extended['err_ES']=df_extended["P"]-df_extended["P^_ES"]
    df_extended['err_semi']=df_extended["P"]-df_extended["P^_semi"]
    df_extended['err_bb']=df_extended["P"]-df_extended["P^_bb"]
    
    #reorder dataframe columns
    new_order=['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb', 'T_a', 'RH', 'Wv', 'atmP', 'G', 's_Wa', 'c_Wa', 's_H', 'c_H',
           's_D', 'c_D', 'dayType', 'ID']
    df_extended=df_extended[new_order]
    
    #calculate correlation
    df_corr=df_extended.drop("ID", axis=1).corr()
    
    #select features to be displayed in the reduced correlation heat map
    reduced_df_corr=df_corr.loc[['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb'], ['P', 'P^_ES', 'P^_semi', 'P^_bb',  'err_ES','err_semi', 'err_bb', 'T_a', 'RH', 'Wv', 'atmP', 'G', 's_Wa', 'c_Wa', 's_H', 'c_H',
       's_D', 'c_D', 'dayType']]
    sns.heatmap(reduced_df_corr, vmin=0.0, vmax=1.0, ax=ax, cbar=False)
    if (i in plots_per_row*np.linspace(0, len(IDs),1))==False:
        ax.set_yticks([])
    if i < len(IDs)-plots_per_row:
        ax.set_xticks([])

    ax.set_title(caseStudy, fontsize=font_size)
    
plt.suptitle("Correlation matrix from all the case studies")
plt.tight_layout()